In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/My Drive/Capstone/Capstone Project/Capstone Data with claims/

/content/drive/My Drive/Capstone/Capstone Project/Capstone Data with claims


In [3]:
import pandas as pd
import ast
import pickle
from sklearn.decomposition import TruncatedSVD

In [4]:
word_mat = pd.read_pickle('allyears_all.pkl')

In [5]:
def convert_Description(text):
    text = text.replace("dict_items(","")
    text = ast.literal_eval(text[:-1])
    return text

In [6]:
def extract_words(text):
  return [word[0] for word in text]

In [7]:
year = 2018
df_lengths = [1,150456,307779,469536,628516,751018]
stop_words = []
print('=================================================================')

for j in range(5):
  print('We are processing the year ',int(year+j),'...')
  
  df = pd.read_csv(str(int(year+j))+'-data-merged.csv')
  df["Description"] = df["Description"].apply(convert_Description)
  df["Words"] = df["Description"].apply(extract_words)
  df.drop(columns = [ 'Unnamed: 0' , 'Doc_number' , 'Title' , 'Type',
                     'App_Date' , 'Country' , 'Org_Name' , 'Org_City' ,
                     'Org_Country' , 'No_inventors' , 'Abstract'] ,
          inplace = True)
  
  temp_word_mat = word_mat.T.iloc[df_lengths[j]-1:df_lengths[j+1]].T
  temp_word_mat = temp_word_mat.drop(['first', 'second', 'say', 'one', 'wherein', 
                          'accord', 'device', 'method', 'comprise','comprising',
                          'comprises', 'include', 'portion', 'least','determine',
                          'select','position','set','use','end'],axis=0)
  dictionary = list(temp_word_mat.T.columns)
  n_components=8
  svd_model = TruncatedSVD(n_components=n_components, algorithm='randomized', n_iter=100, random_state=122)
  print('Running LSA model...')
  lsa = svd_model.fit_transform(temp_word_mat.T)
  print('LSA model finished.')
  index = [str('topic_' + str(num)) for num in range(n_components)]
  encoding_matrix = pd.DataFrame(svd_model.components_, index = index, columns = (dictionary)).T

  topic_encoded_df = pd.DataFrame(lsa, columns = index)
  topic_encoded_df["Description"] = df['Description']
  topic_encoded_df["Best_topic"] = topic_encoded_df[index].idxmax(axis=1)
  topic_encoded_df.to_csv('Topics-'+str(int(year+j))+'-data-merged.csv',index=False)
  sets = set()
  for i in encoding_matrix.columns:
    print(f'Top weighing words for {i} as below:')
    temp = encoding_matrix.sort_values(by=[i],ascending=False, key=abs)
    w=0
    while w<len(temp) and (temp[i][w]>=0.1 or temp[i][w]<=-0.1):
        w+=1
    print([str(str(x)+"*"+str(temp[i][x])) for x in temp[i][:w].index])
    print()
    tempset= set()
    if i=='topic_1':
      for x in temp[i][:w].index:
        tempset.add(str(x))
      sets=tempset
    else:
      for x in temp[i][:w].index:
        tempset.add(str(x))
    sets.intersection(tempset)
  stop_words.append(list(sets))
  print('Possible stop words here: ', stop_words[j])
  print('=================================================================')

We are processing the year  2018 ...
Running LSA model...
LSA model finished.
Top weighing words for topic_0 as below:
['system*0.24496665616663804', 'configure*0.23619633578766233', 'data*0.23582071558480602', 'plurality*0.22027708743113492', 'base*0.1877808218779788', 'far*0.1845159813564128', 'signal*0.1813601449663799', 'surface*0.15156977094989188', 'id*0.1406953603099802', 'control*0.13288258058141464', 'value*0.13126749935171012', 'information*0.13047693812391997', 'processor*0.1256668672031873', 'medium*0.121517396519798', 'computer*0.12044778243770149', 'user*0.11922303181068392', 'receive*0.11765157484307716', 'element*0.11431311944001057', 'time*0.10956277098289445', 'instruction*0.10368621586826939', 'layer*0.10180140137401571', 'unit*0.10073717104360883']

Top weighing words for topic_1 as below:
['id*0.9355618720384107', 'region*0.16333157898419318', 'light*0.14789545232533924', 'acid*0.128828791405795', 'sequence*0.11937407384990964']

Top weighing words for topic_2 as b